In [12]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import OrderedDict
from CKA_functions import adjacency_matrix_motion,adjacency_matrix_distance_motion
from CKA_functions import adjacency_matrix_FACED,adjacency_matrix_distance_FACED
import math
import importlib
import SGCN_FACED_norm
importlib.reload(SGCN_FACED_norm)
from SGCN_FACED_norm import ShallowSGCNNet
from torchinfo import summary

In [13]:
in_chans = 32
n_classes = 3
input_window_samples = 400

adj_m,pos = adjacency_matrix_FACED()
#print(adj_m)
adj_dis_m, dm = adjacency_matrix_distance_FACED(pos,delta=6)
dm
torch_tensor = torch.from_numpy(dm)
edge_weight = torch_tensor.reshape(-1)
print(edge_weight.shape)
model = ShallowSGCNNet(in_chans,n_classes,input_window_samples,edge_weight)

torch.Size([1024])


In [14]:
print(model)

ShallowSGCNNet(
  (temporal): Conv2d(1, 20, kernel_size=(1, 25), stride=(1, 1))
  (tpool): AvgPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0)
  (tbatch_norm): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sgconv): SimpleGCNNet(
    (sgconv): SGConv(188, 20, K=2)
  )
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (batch_norm): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=640, out_features=3, bias=True)
)


In [15]:

# Define a threshold distance below which nodes will be connected
threshold = 0  # Adjust as needed

source_nodes = []
target_nodes = []

# Iterate over all elements in the distance matrix, including self-loops and duplicates
for i in range(dm.shape[0]):
    for j in range(dm.shape[1]):  # Iterate over all pairs, including (i, i)
        if dm[i, j] >= threshold:  # If the distance meets the condition
            source_nodes.append(i)  # Source node
            target_nodes.append(j)  # Target node

# Create the edge_index tensor
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

print("Distance Matrix Shape:", dm.shape)
print("Edge Index:\n", edge_index.shape)

Distance Matrix Shape: (32, 32)
Edge Index:
 torch.Size([2, 1024])


In [16]:
print(edge_index.shape)

torch.Size([2, 1024])


In [ ]:
adj_matrix = model.sgconv.edge_weights.detach().cpu().numpy().reshape(32,32)
fig, ax = plt.subplots(figsize=(8, 8))
cax = ax.matshow(adj_matrix, cmap='viridis', interpolation='nearest',vmin=0, vmax=1)  # Adjust color map
fig.colorbar(cax)
ax.set_title("Learned Edge Weights (Adjacency Matrix)")
ax.set_xlabel("Node Index")
ax.set_ylabel("Node Index")


# Finally, show the plot
plt.show()

ValueError: cannot reshape array of size 1024 into shape (22,22)

In [ ]:
summary(model,input_size=(2, 22, 1125),edge_index=edge_index, col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowSGCNNet                           [2, 22, 1125]             [2, 4]                    --                        --
├─Conv2d: 1-1                            [2, 1, 22, 1125]          [2, 40, 22, 1101]         1,040                     [1, 25]
├─BatchNorm2d: 1-2                       [2, 40, 22, 1101]         [2, 40, 22, 1101]         80                        --
├─AvgPool2d: 1-3                         [2, 40, 22, 1101]         [2, 40, 22, 550]          --                        [1, 2]
├─SimpleGCNNet: 1-4                      [2, 40, 22, 550]          [2, 40, 22, 10]           484                       --
│    └─SGConv: 2-1                       [2, 40, 22, 550]          [2, 40, 22, 10]           --                        --
│    │    └─SumAggregation: 3-1          [2, 40, 484, 550]         [2, 40, 22, 550]          --                        --
│    